In [1]:
import requests
import json
import os
from urllib.parse import urlencode
from utilities import logger
import mysql.connector
#from utilities import ConfigReader
from utilities.sha_tools import generate_sha_digest
from sqlalchemy import Column, Integer, String, Text, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import time
from bs4 import BeautifulSoup

api_user = 'h=Czxr?m'

api_password = 'B537AF1C-64F9-FB2D-DC31-1E8C334E8D79'
mylogger = logger.Logger(name='TAPD_ana', debug=True).logger

def html_to_text(html):
    if not html or len(html) == 0:
        return ''
    
    #"抛弃HTML标签，只提取所有文本内容"
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text()
    return text


In [2]:
# 创建基础类
Base = declarative_base()

username = 'root'
password = 'newpass'
#host = "127.0.0.1"
host = '10.101.9.50'
dbname = 'requirements'
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{dbname}")
Session = sessionmaker(bind=engine)
local_session = Session()

try:
    # 连接数据库，并执行一个简单查询
    connection = engine.connect()
    result = connection.execute("SELECT 1")
    
    # 打印结果
    mylogger.debug(f"Database connection successful. Result: {result.scalar()}")
    
    # 不要忘记关闭连接！
    connection.close()
    
except Exception as e:
    mylogger.error(f"Database connection failed. Error: {e}")

class Project(Base):
    __tablename__ = 'project_info'

    workspace_id = Column(String(255), primary_key=True)
    name = Column(String(255))
    systems = Column(String(255))
    modify_time = Column(Date)
    extension1 = Column(String(255))
    url = Column(String(255))
    
class BaseStory(Base):
    __tablename__ = 'base_stories'
    __abstract__ = True
    
    id = Column(Integer, primary_key=True)
    workspace_id = Column(String(255))
    story_id = Column(String(255))
    name = Column(String(512))
    description = Column(Text)
    url = Column(String(255))
    status = Column(String(255))
    developer = Column(String(255))
    creator = Column(String(255))
    finger_print = Column(String(255))
    extension1 = Column(String(255))
    extension2 = Column(String(255))
    extension3 = Column(String(255))
    extension4 = Column(String(255))
    content = Column(Text)
    vector_id = Column(Integer)
    modify_time = Column(Date)
    close_time = Column(Date)
class Story(BaseStory):
    __tablename__ = 'stories'    


class BPStory(BaseStory):
    __tablename__ = 'stories_bp' 
    customer = Column(String(255))

2024-03-24 22:46:21.999 | DEBUG    | __main__:<module>:19 - Database connection successful. Result: 1


In [7]:
import pandas as pd
from datetime import datetime

data = pd.read_excel('project-0129.xlsx')

# admin info is added in "项目信息.xlsx"，this function is not longer used
def update_admin():
        for _, row in data.iterrows():
                project_record = local_session.query(Project).filter_by(workspace_id=row['workspace_id']).first()

                # 更新extension字段
                if project_record:
                        project_record.extension1 = row['管理员']
                        local_session.commit()
                        print(f"Record with workspace_id '{project_record.workspace_id}' updated successfully.")
                else:
                        print(f"Record with workspace_id 'your_workspace_id' not found.")

In [3]:
#from sqlalchemy import create_engine, Table, MetaData
#from sqlalchemy.orm import sessionmaker
import pandas as pd
from datetime import datetime

current_datetime = datetime.now()
current_date = current_datetime.date()

# 使用pandas读取excel文件
data = pd.read_excel('项目信息.xls')

execute_add_flag = True

# 遍历数据行并插入到数据库表中
for _, row in data.iterrows():
    if not local_session.query(Project).filter_by(workspace_id=row['workspace_id']).first():  
        mylogger.debug(f"need to add project {row['name']}")      
        if execute_add_flag:
                record={'workspace_id':row['workspace_id'], 
                        'name':row['name'], 
                        'systems':row['systems'], 
                        'modify_time':current_date, 
                        'url':f"https://www.tapd.cn/{row['workspace_id']}",
                        'extension1':row['admin']}
                project = Project(**record)
                local_session.add(project)  

# 提交会话
if execute_add_flag:
      local_session.commit()

In [5]:
# 查询所有的workspace_id
workspace_ids = local_session.query(Project.workspace_id).all()
# 将结果转为列表
workspace_id_list = [id[0] for id in workspace_ids]


wid_already_in = local_session.query(Story.workspace_id).distinct().all()
wid_already_in_list = [id[0] for id in wid_already_in]
wid_already_in_list.append('62366085') # BP，单独存储
wid_already_in_list.append('39451937') # 星票通，单独存储
wid_already_in_list.append('54057645') #雅迪，没有需求
new_ids = set(workspace_id_list).difference(set(wid_already_in_list))
mylogger.debug(len(new_ids))

2024-03-24 22:46:52.747 | DEBUG    | __main__:<module>:13 - 5


In [4]:
story_table = '../stories'
def isBP():
    if 'bp' in story_table:
        return True
    else:
        return False
isBP()    

False

In [6]:
def insert_data(session, record):
    story = None
    if isBP():
        if not session.query(BPStory).filter_by(story_id=record['story_id']).first():
            story = BPStory(**record)
    else:
        if not session.query(Story).filter_by(story_id=record['story_id']).first():
            story = Story(**record)
    if story:
        session.add(story)
        session.commit()

In [7]:
from tqdm import tqdm

def update_story_db(items):
    total = len(items)
    for story in tqdm(items, total=total):
        # if story['workspace_id'] not in complete_wid:
        #     continue
        record = {}
        record['workspace_id'] = story['workspace_id']
        record['story_id'] = story['id']
        record['name'] = story['name']
        record['creator'] = story['creator']
        record['status'] = story['status']
        record['developer'] = story['developer']
        if story['description']: 
            record['description'] = story['description'][:3000] 
        else: 
            record['description'] = ''
        record['url'] = f"https://www.tapd.cn/{record['workspace_id']}/prong/stories/view/{record['story_id']}"
        record['content'] = html_to_text(record['description'])
        record['finger_print'] = generate_sha_digest(record['content'])
        record['modify_time'] = story['modified']
        if story['completed']:
            record['close_time'] = story['completed']
        else:
            record['close_time'] = '2100-01-01'
        
        if isBP():        
            record['customer'] = story['custom_field_one']
        
        insert_data(local_session, record)

In [16]:
new_ids=['66563110']

In [17]:
sum = 0
base_url = "https://api.tapd.cn/stories"
page_to_break = 0
item_list_original = []
item_to_update = []
item_to_analyze = []
complete_wid = set()
i = 1

for wid in new_ids:    
    try:
        r = requests.get(f"https://api.tapd.cn/stories/count?workspace_id={wid}", auth=(api_user, api_password))
        ret = r.text # 获取接口返回结果
        # print(ret) # Python 3
        json_string = json.loads(ret)
        total_entries = json_string['data']['count']
        pinfo = local_session.query(Project.name).filter(Project.workspace_id == wid).first()
        prj_name = pinfo[0]
        # mylogger.info(f'{wid} {prj_name}, {total_entries}')
        sum += total_entries
    except Exception as e:
        mylogger.error(f"{wid} {prj_name} Something goes wrong {e}")    
        continue

    current_page = 1    
    if total_entries >= 1000:
        batch_size = 200
    else:
        batch_size =100
    current_index = 0
    item_number = 0    

    current_batch = []
    mylogger.debug(f"{i}/{len(new_ids)} Start to process {wid} {prj_name} {total_entries} entries")
    while current_index < total_entries:
        mylogger.debug(f">>>>>>>>>>Fetch page #{current_page}")
        params = {
            "workspace_id": wid,
            "page": current_page,
            "limit": batch_size
        }

        query_string = urlencode(params)
        full_url = f"{base_url}?{query_string}"
        
        r = requests.get(full_url, auth=(api_user, api_password))
        # r = requests.get(f"https://api.tapd.cn/tapd_wikis?workspace_id={workspace_id}", auth=(api_user, api_password))
        ret = r.text # 获取接口返回结果
        decoded_data = json.loads(ret)
        if(decoded_data['status'] != 1):
            mylogger.error(decoded_data)
            break
        else:
            story_entries = decoded_data["data"]
            #mylogger.debug(f"number of wiki pages are {len(story_entries)}")
            index = 1
            for entry in story_entries:
                story = entry["Story"]
                item_to_analyze.append(story)
                result = local_session.query(Story).filter(Story.story_id == story['id']).first()
                # sql = f"SELECT story_id, finger_print FROM {story_table} WHERE story_id=%s"
                # execute_query(sql, (story["id"],))
                # results = db_cursor.fetchone()            
                if result:
                    ID = result.story_id
                    finger_print =result.finger_print    
                    sha_digest = generate_sha_digest(html_to_text(story["description"]))
                    if finger_print == sha_digest:
                        #mylogger.debug(f"story {ID} already exist, content is the same")
                        pass
                    else:
                        #mylogger.debug(f"story {ID} already exist, but content is different")  
                        item_to_analyze.append(story)             
                        current_batch.append(story)
                else:
                    #mylogger.info(f"{story['id']} {story['name']} should be added")
                    item_to_update.append(story)   
                    current_batch.append(story)
            current_index += batch_size
            current_page += 1
            page_to_break += 1
            # if current_page >= 2:
            #     break
            time.sleep(1)
            if current_page % 10 == 0:
                time.sleep(20)  

    update_story_db(current_batch)
    complete_wid.add(wid)  
    i += 1
    time.sleep(5) 
                 
mylogger.debug(f'total entries = {sum}')        

2024-03-24 23:05:22.300 | DEBUG    | __main__:<module>:34 - 1/1 Start to process 66563110 国家电投 3335 entries
2024-03-24 23:05:22.302 | DEBUG    | __main__:<module>:36 - >>>>>>>>>>Fetch page #1
2024-03-24 23:06:03.344 | DEBUG    | __main__:<module>:36 - >>>>>>>>>>Fetch page #2
2024-03-24 23:06:42.618 | DEBUG    | __main__:<module>:36 - >>>>>>>>>>Fetch page #3
2024-03-24 23:07:22.087 | DEBUG    | __main__:<module>:36 - >>>>>>>>>>Fetch page #4
2024-03-24 23:08:01.914 | DEBUG    | __main__:<module>:36 - >>>>>>>>>>Fetch page #5
2024-03-24 23:08:41.161 | DEBUG    | __main__:<module>:36 - >>>>>>>>>>Fetch page #6
2024-03-24 23:09:20.925 | DEBUG    | __main__:<module>:36 - >>>>>>>>>>Fetch page #7
2024-03-24 23:10:01.632 | DEBUG    | __main__:<module>:36 - >>>>>>>>>>Fetch page #8
2024-03-24 23:10:42.874 | DEBUG    | __main__:<module>:36 - >>>>>>>>>>Fetch page #9
2024-03-24 23:11:43.219 | DEBUG    | __main__:<module>:36 - >>>>>>>>>>Fetch page #10
2024-03-24 23:12:23.350 | DEBUG    | __main__:<modu

 80%|████████  | 2681/3335 [07:14<02:41,  4.05it/s]C:\Users\NstcUser\AppData\Local\Temp\ipykernel_16808\1660417174.py:26: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, "html.parser")
100%|██████████| 3335/3335 [09:36<00:00,  5.79it/s]


2024-03-24 23:26:33.092 | DEBUG    | __main__:<module>:93 - total entries = 3335


In [13]:
complete_wid

{'39451937'}

### 以下任务在server上运行，作废

In [15]:
import faiss
import numpy as np
import shutil
from tqdm import tqdm 

if isBP():
    #total_stories = local_session.query(BPStory).filter(BPStory.vector_id == None).all()
    total_stories = local_session.query(BPStory).filter(BPStory.vector_id == None).order_by(BPStory.id.asc()).all()
    index = faiss.read_index("bp_story.faiss")
    # index = faiss.IndexFlatL2(1024)
else:
    total_stories = local_session.query(Story).filter(Story.vector_id == None).all()
    index = faiss.read_index("../story.faiss")
    
length = len(total_stories)
mylogger.debug(f'to process: {len(total_stories)}, size of FAISS: {index.ntotal}')

2024-01-26 10:03:50.042 | DEBUG    | __main__:<module>:16 - to process: 908, size of FAISS: 17673


In [16]:
first_file_flg = True
for story in tqdm(total_stories,total=length):  
    if story.vector_id is None or story.vector_id == '':  # 检查vector_id是否为空字符串或None  
        # mylogger.debug(f"Start to process {index.ntotal}/{len(local_session.query(Story).all())}")
        if isBP():
            content = story.name
        else:
            content = '标题：' + story.name + "\n" + '内容：' + story.content
        vec = create_embedding_bge(content)
        story.vector_id = index.ntotal
        index.add(np.array([vec]))    
        local_session.add(story)  # 将更新后的故事添加回session，这样更改才会被保存到数据库中    
        
    if index.ntotal%25 == 0:
        if isBP():
            file_name = f'bp_story_{index.ntotal}.faiss'
        else:
            file_name = f'story_{index.ntotal}.faiss'
        local_session.commit()
        faiss.write_index(index, file_name)
        
        if first_file_flg:
            first_file_flg = False
        else:
            os.remove(last_file_name)
        last_file_name = file_name  
            
local_session.commit()
if isBP():
    faiss.write_index(index, "bp_story.faiss")
else:
    faiss.write_index(index, "story.faiss")

100%|██████████| 908/908 [17:45<00:00,  1.17s/it]


In [17]:
story_index = faiss.read_index('bp_story.faiss')
story_index.ntotal

18581

In [ ]:
current_page = 1
base_url = "https://api.tapd.cn/stories"
if total_entries >= 1000:
    batch_size = 200
else:
    batch_size =100
current_index = 0
item_number = 0
item_list_original = []
item_to_update = []
item_to_analyze = []


while current_index < total_entries:
    mylogger.debug(f">>>>>>>>>>Fetch page #{current_page}")
    params = {
        "workspace_id": workspace_id,
        "page": current_page,
        "limit": batch_size
    }

    query_string = urlencode(params)
    full_url = f"{base_url}?{query_string}"
    
    r = requests.get(full_url, auth=(api_user, api_password))
    # r = requests.get(f"https://api.tapd.cn/tapd_wikis?workspace_id={workspace_id}", auth=(api_user, api_password))
    ret = r.text # 获取接口返回结果
    decoded_data = json.loads(ret)
    if(decoded_data['status'] != 1):
        mylogger.error(decoded_data)
        break
    else:
        story_entries = decoded_data["data"]
        mylogger.debug(f"number of wiki pages are {len(story_entries)}")
        index = 1
        for entry in story_entries:
            story = entry["Story"]
            item_to_analyze.append(story)
            sql = f"SELECT story_id, finger_print FROM {story_table} WHERE story_id=%s"
            execute_query(sql, (story["id"],))
            results = db_cursor.fetchone()            
            if results:
                ID, finger_print = results                
                sha_digest = generate_sha_digest(html_to_text(story["description"]))
                if finger_print == sha_digest:
                    #mylogger.debug(f"story {ID} already exist, content is the same")
                    pass
                else:
                    #mylogger.debug(f"story {ID} already exist, but content is different")  
                    item_to_analyze.append(story)             
            else:
                #mylogger.info(f"{story['id']} {story['name']} should be added")
                item_to_update.append(story)   
        current_index += batch_size
        current_page += 1
        # if current_page >= 2:
        #     break
        time.sleep(1)
        if current_page % 10 == 0:
            time.sleep(20)

In [6]:
workspace_id = '62366085'
project_name = {'55368532':'湖北交投司库',
        '41574258':'产品研发',
        '31121576':'蔚来汽车司库',
        '50719964':'中国有色司库',
        '37502023':'中建科司库', 
        '46779253':'陕投司库',
        '49978871':'东风司库',
        '56370180':'山子高科G20',
        '65731238':'正浩创新G20',
        '52867360':'坚果投影G20',
        '42614505':'瑞派宠物医院G20',
        '30386009':'深信服G20',
        '54830814':'九号公司G20',
        '62366085':'BP'}

{"status":1,"data":{"count":17688},"info":"success"}
2024-01-23 19:00:25.724 | INFO     | __main__:<module>:23 - 17688


In [36]:
mylogger.debug(html_to_text(item_to_analyze[0]['description']))

2024-01-20 17:57:21.253 | DEBUG    | __main__:<module>:1 - 详细方案见附件


In [18]:
from tqdm import tqdm
total = len(item_to_update)

for story in tqdm(item_to_update, total=total):
    if story['workspace_id'] not in complete_wid:
        continue
    record = {}
    record['workspace_id'] = story['workspace_id']
    record['story_id'] = story['id']
    record['name'] = story['name']
    record['creator'] = story['creator']
    record['status'] = story['status']
    record['developer'] = story['developer']
    if story['description']: 
        record['description'] = story['description'][:3000] 
    else: 
        record['description'] = ''
    record['url'] = f"https://www.tapd.cn/{workspace_id}/prong/stories/view/{record['story_id']}"
    record['content'] = html_to_text(record['description'])
    record['finger_print'] = generate_sha_digest(record['content'])
    record['modify_time'] = story['modified']
    if story['completed']:
        record['close_time'] = story['completed']
    else:
        record['close_time'] = '2100-01-01'
    
    if isBP():        
        record['customer'] = story['custom_field_one']
    
    insert_data(local_session, record)

  0%|          | 0/15221 [00:00<?, ?it/s]

In [36]:
local_session.close()

In [17]:
if isBP():
    story_index = faiss.read_index('bp_story.faiss')
else:
    story_index = faiss.read_index('story.faiss')

def query(question, limit=10):
    query_vec = create_embedding_bge(question)
    distances, indices = story_index.search(np.array([query_vec], dtype=np.float32), limit)

    results = []
    for i in range(limit):
        index = indices[0][i]
        distance = distances[0][i]      
        if isBP():
            query_condition = BPStory.vector_id == index               
            res = local_session.query(BPStory).filter(query_condition).all()  
        else:
            query_condition = Story.vector_id == index           
            res = local_session.query(Story).filter(query_condition).all()          
        
        mylogger.debug(f"Result {i + 1}> Distance:{distance}, index:{index}, Content:{res[0].name}")      
        results.append(res[0])
        
    return results

In [14]:
story_index.ntotal

17673

In [ ]:
question ='功能是在预算模块增加多个预算组，有类似的功能实现吗？'
question = '麻烦解释一下招商银行历史余额查询交易'
results = query(question)

for idx, result in enumerate(results, start=1):
    res_str = f'[{idx}] project: {project_name[result.workspace_id]}'
    res_str += f'; creator：{result.creator}'
    res_str += f'; subject: {result.name}'
    if isBP():
        res_str += f'\nmtime: {result.modify_time}, ctime: {result.close_time}, customer: {result.customer}'
    else:
        res_str += f'\n content: {result.content}'    
    res_str += f'\n URL: {result.url}\n'
    res_str += f'-'*60
    mylogger.debug(res_str)


In [ ]:
import requests
url = 'http://localhost:5003/query'
input_string = '付款模块支付单可以修改付方账号'
response = requests.post(url, json={'input_string': input_string}) 
mylogger.debug(response)

In [ ]:
response.json()